# Dependencies

## Imports

In [1]:
import numpy as np
import pandas as pd
import datetime as dt

import plotly.plotly as py
import plotly.graph_objs as go

# uses 'orca' to be able to save plotly images to a file
import plotly.io as pio

# allows for offline plotly graphing in jupyter notebook
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect, between

## Connect db to session via SQLAlchemy 

In [3]:
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

# reflect an existing database into a new model
Base = automap_base()

# reflect the tables in the 'hawaii.sqlite' file
Base.prepare(engine, reflect=True)

# We can view all of the classes that automap found
Base.classes.keys()

['measurement', 'station']

## Creating References to the tables

In [4]:
# reference to the 'measurement' table
Measurements = Base.classes.measurement

# reference to the 'station' table
Station = Base.classes.station

# linking the 'engine'/db to the jupyter notebook session
session = Session(engine)

# Climate Analysis

## Retrieve Precipitation Data

In [5]:
# chose to retrieve data from 2016
rain_measurements = session.query(Measurements.date, Measurements.prcp).filter(between(
    Measurements.date, '2016-01-01', '2016-12-31')).order_by(Measurements.date.asc())

print(rain_measurements)

SELECT measurement.date AS measurement_date, measurement.prcp AS measurement_prcp 
FROM measurement 
WHERE measurement.date BETWEEN ? AND ? ORDER BY measurement.date ASC


In [6]:
# saved the query to a variable using '.all()' and a variable
rain_data = rain_measurements.all()

# save the 'rain data' variable/query as df
rain_df = pd.DataFrame(rain_data)

# replacing all NaN values with '0'
rain_df = rain_df.fillna(value=0)

rain_df.head()

,date,prcp
0,2016-01-01,0.00
1,2016-01-01,0.02
2,2016-01-01,0.00
3,2016-01-01,0.00
4,2016-01-01,0.02


## Plot Precipitation Data

In [7]:
# Create a trace via plotly
trace = go.Scatter(
    x = rain_df.date,
    y = rain_df.prcp,
)

data = [trace]

# Edit the layout
layout = dict(title = '2016 Percipitation Data',
              xaxis = dict(title = 'Date'),
              yaxis = dict(title = 'Percipitation (inches)'),
              )

fig = dict(data=data, layout=layout)

# iplot allows for offline Plotly graphing
iplot(fig, filename='basic-line')

In [8]:
# saved the '2016' Percipitation Data as a file
pio.write_image(fig, 'Images/percipitation-data.png')

In [9]:
# Use Pandas to calculate the summary statistics for the precipitation data
rain_summary = rain_df.describe()
rain_summary

,prcp
count,2309.000000
mean,0.161152
std,0.482446
min,0.000000
25%,0.000000
50%,0.010000
75%,0.120000
max,9.640000


# Stations Analysis

## Station Count Query

In [10]:
# Design a query to show how many rows are available in this dataset
station_count = session.query(Station).count()

print(f'There are {station_count} stations in the table.')

There are 9 stations in the table.


## Retrieve Active Station Data

In [11]:
# queried station data, counted the amount of times a station appeared using 'func.count()'
stations = session.query(Measurements.station, func.count(
    Measurements.station)).group_by(Measurements.station).order_by(func.count(Measurements.station).desc())

print(stations)

SELECT measurement.station AS measurement_station, count(measurement.station) AS count_1 
FROM measurement GROUP BY measurement.station ORDER BY count(measurement.station) DESC


In [12]:
# saved the query to a variable using '.all()' and a variable
station_count = stations.all()

station_count

[('USC00519281', 2772),
 ('USC00519397', 2724),
 ('USC00513117', 2709),
 ('USC00519523', 2669),
 ('USC00516128', 2612),
 ('USC00514830', 2202),
 ('USC00511918', 1979),
 ('USC00517948', 1372),
 ('USC00518838', 511)]

## Retrieve TOBS Data

In [13]:
# retrieved data from 2016, from the most active station 'USC00519281'
tobs_measurements = session.query(Measurements.station, Measurements.tobs, Measurements.date).filter(between(
    Measurements.date, '2016-01-01', '2016-12-31')).filter(Measurements.station == "USC00519281")

print(tobs_measurements)

SELECT measurement.station AS measurement_station, measurement.tobs AS measurement_tobs, measurement.date AS measurement_date 
FROM measurement 
WHERE measurement.date BETWEEN ? AND ? AND measurement.station = ?


In [14]:
# saved the query to a variable using '.all()' and a variable
tobs_data = tobs_measurements.all()

# save the 'tobs data' variable/query as df
tobs_df = pd.DataFrame(tobs_data)

# replacing all NaN values with '0' if existing
tobs_df = tobs_df.fillna(value=0)

# lowest tobs=56, highest tobs=82
tobs_df.head()

,station,tobs,date
0,USC00519281,74.0,2016-01-01
1,USC00519281,72.0,2016-01-02
2,USC00519281,68.0,2016-01-03
3,USC00519281,69.0,2016-01-04
4,USC00519281,70.0,2016-01-05


## Plot TOBS Data

In [15]:
# create a histogram via plotly
x = tobs_df.tobs


data = [go.Histogram(x=x)]


# Edit the layout
layout = dict(title = 'Observation Data from Station USC00519281',
              xaxis = dict(title = 'TOBS Measurement'),
              yaxis = dict(title = 'Frequency of Days with TOBS'),
              )

fig = go.Figure(data=data, layout=layout)

# iplot allows for offline Plotly graphing
iplot(fig, filename='basic histogram')


In [16]:
# saved the 'Tobs' frequency data as a file
pio.write_image(fig, 'Images/tobs-data.png')

# Temperature Analysis

## Retrieve Temperature Data

In [17]:
# This function called `calc_temps` will accept start date and end date in the format '%Y-%m-%d'
# and return the minimum, average, and maximum temperatures for that range of dates


def calc_temps(start_date, end_date):
    """TMIN, TAVG, and TMAX for a list of dates.

    Args:
        start_date (string): A date string in the format %Y-%m-%d
        end_date (string): A date string in the format %Y-%m-%d

    Returns:
        TMIN, TAVE, and TMAX
    """

    return session.query(func.min(Measurements.tobs), func.avg(Measurements.tobs), func.max(Measurements.tobs)).\
        filter(Measurements.date >= start_date).filter(
            Measurements.date <= end_date).all()

In [18]:
# Use your previous function `calc_temps` to calculate the tmin, tavg, and tmax 
# for your trip using the previous year's data for those same dates.

temp_last_yr = calc_temps('2015-01-01', '2015-12-31')
print(temp_last_yr)

for temp in temp_last_yr:
    min_temp = temp[0]
    avg_temp = temp[1]
    max_temp = temp[2]

[(56.0, 73.8603305785124, 86.0)]


## Plot Temperature Data

In [19]:
# create a trace via plotly
trace = go.Bar( 
    y=[avg_temp],
    name='Trip Avg Temp',
    error_y=dict(
        type='data',
        array=[min_temp, max_temp],
        visible=True,
        width = 8,
        thickness = 3,
    ))

# saving the 'trace' as the data variable
data = [trace]

# Edit the layout
layout = dict(title='Trip Average Temperature',
              xaxis=dict(
                  visible=False,
                  type="category"
              ),
              yaxis=dict(
                  title='Temperature (F)',
                  range=[0,95]
              ))

# creating a 'fig' variable for the data,layout variables
fig = go.Figure(data=data, layout=layout)

# iplot allows for offline Plotly graphing
iplot(fig, filename='error-bar-bar')


In [20]:
# saved the '2016' Percipitation Data as a file
pio.write_image(fig, 'Images/average-temperature-data.png')

# Rainfall Cummulation Analysis

## Retrieve Rainfall Cummulative Data

In [24]:
# Calculate the rainfall per weather station for your trip dates using the previous year's matching dates.
# Sort this in descending order by precipitation amount and list the station, name, latitude, longitude, and elevation

rainfall = session.query(Measurements.station, func.sum(
    Measurements.prcp)).filter(Measurements.date >= '2015-01-01').filter(
            Measurements.date <= '2015-12-31').group_by(Measurements.station).order_by(func.sum(Measurements.prcp).desc())

print(rainfall)

SELECT measurement.station AS measurement_station, sum(measurement.prcp) AS sum_1 
FROM measurement 
WHERE measurement.date >= ? AND measurement.date <= ? GROUP BY measurement.station ORDER BY sum(measurement.prcp) DESC


In [44]:
# saved the query to a variable using '.all()' and a variable
rainfall_data = rainfall.all()

# created a df of the query to place in the bar chart
rainfall_df = pd.DataFrame(rainfall_data, columns=['station', 'tot_rainfall'])

## Plot Rainfall Cummulation Data

In [52]:
trace = go.Bar(
    x=rainfall_df['station'],
    y=rainfall_df['tot_rainfall'],
    name='Cummulative Rainfall',
    )


data = [trace]

layout = go.Layout(
    title = 'Rain Cummulative Data',
    yaxis = dict(title = 'Rainfall (inches)'),
    xaxis= dict(tickangle=-45)
)




fig = go.Figure(data=data, layout=layout)
iplot(fig, filename='angled-text-bar')